# Route between multiple Runnables

This notebook covers how to do routing in the LangChain Expression Language.

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

There are two ways to perform routing:

1. Using a `RunnableBranch`.
2. Writing custom factory function that takes the input of a previous step and returns a **runnable**. Importantly, this should return a **runnable** and NOT actually execute.

We'll illustrate both methods using a two step sequence where the first step classifies an input question as being about `LangChain`, `Anthropic`, or `Other`, then routes to a corresponding prompt chain.

## Using a RunnableBranch

A `RunnableBranch` is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it's invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input. 

If no provided conditions match, it runs the default runnable.

Here's an example of what it looks like in action:

In [1]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.schema.output_parser import StrOutputParser

First, let's create a chain that will identify incoming questions as being about `LangChain`, `Anthropic`, or `Other`:

In [2]:
chain = PromptTemplate.from_template("""Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.
                                     
Do not respond with more than one word.

<question>
{question}
</question>

Classification:""") | ChatAnthropic() | StrOutputParser()

In [3]:
chain.invoke({"question": "how do I call Anthropic?"})

' Anthropic'

Now, let's create three sub chains:

In [4]:
langchain_chain = PromptTemplate.from_template("""You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:""") | ChatAnthropic()
anthropic_chain = PromptTemplate.from_template("""You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:""") | ChatAnthropic()
general_chain = PromptTemplate.from_template("""Respond to the following question:

Question: {question}
Answer:""") | ChatAnthropic()

In [5]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
  (lambda x: "anthropic" in x["topic"].lower(), (lambda x: {"question": x["question"]}) | anthropic_chain),
  (lambda x: "langchain" in x["topic"].lower(), (lambda x: {"question": x["question"]}) | langchain_chain),
  (lambda x: {"question": x["question"]}) | general_chain
)

In [6]:
full_chain = {
    "topic": chain,
    "question": lambda x: x["question"]
} | branch

In [7]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content=" As Dario Amodei told me, here is how you can use Anthropic:\n\n1. Sign up for an account on the Anthropic website. This will allow you to access Anthropic's AI assistant Claude and start interacting with it.\n\n2. Try asking Claude general knowledge questions or having natural conversations to see its conversational abilities. You can ask Claude about topics like sports, movies, current events, math problems, etc.\n\n3. Use Claude for writing assistance. Describe what you want written and Claude will generate draft text for you. You can give it prompts for emails, articles, stories and more. \n\n4. Leverage Claude's capabilities for content generation. Provide it with a topic and Claude can create original text around it. This can be helpful for research, articles, presentations and more.\n\n5. Integrate Claude into your own applications using Anthropic's API. This allows you to build customized AI solutions with Claude's natural language processing skills.\n\n6. Ch

In [8]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content=" As Harrison Chase told me, here are a few tips for using LangChain:\n\nFirst, think carefully about what you want LangChain to do. Ask clear, specific questions that provide necessary context and details. The more information you give LangChain, the better it can understand and assist you. \n\nWhen entering your question or request, be polite and respectful. LangChain responds best to thoughtful, good faith inquiries. Avoid harmful, unethical, dangerous, or illegal requests.\n\nStart simply. Ask basic questions to get a feel for LangChain's capabilities before attempting more advanced or nuanced requests. Build up complexity gradually.\n\nBe patient. LangChain may take some time to generate thoughtful, high-quality responses to open-ended or ambiguous questions. Allow it a moment to compose its answers.\n\nProvide feedback. Let LangChain know when its responses are helpful, harmful, or off-base. This feedback helps it improve. Upvote useful responses and downvote un

In [9]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content=' 4', additional_kwargs={}, example=False)

## Using a custom function

You can also use a custom function to route between different outputs. Here's an example:

In [10]:
def route(info):
    inputs = {"question": lambda x: x["question"]}
    if "anthropic" in info["topic"].lower():
        return inputs | anthropic_chain
    elif "langchain" in info["topic"].lower():
        return inputs | langchain_chain
    else:
        return inputs | general_chain

In [11]:
from langchain.schema.runnable import RunnableLambda

full_chain = {
    "topic": chain,
    "question": lambda x: x["question"]
} | RunnableLambda(route)

In [14]:
full_chain.invoke({"question": "how do I use Anthroipc?"})

AIMessage(content=" As Dario Amodei told me, to use Anthropic's conversational AI system Anthropic Intelligent Personal Assistant (Anthro IPC), you simply need to chat with it through the web interface or API. Anthro IPC is designed to have natural conversations and be helpful, harmless, and honest.", additional_kwargs={}, example=False)

In [13]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content=" As Harrison Chase told me, here is how you use LangChain:\n\nLangChain is an AI assistant that can have natural conversations and provide helpful information. To use LangChain, simply chat with it in a conversational way, asking questions or making requests. It will try to understand what you are asking and provide an appropriate response. \n\nYou can ask LangChain open-ended questions, and it will generate thoughtful answers drawing from its broad knowledge. Or you can ask it for specifics, like definitions of words, calculations, or facts about certain topics. LangChain aims to have nuanced, high-quality conversations spanning many subjects.\n\nSome tips for using LangChain effectively:\n\n- Ask clear, straightforward questions and state requests plainly. Don't use too much slang or figures of speech that could confuse it.\n\n- You can follow up on its responses by asking for clarification, additional details, examples, etc. Having a back-and-forth conversation he

In [15]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content=' 4', additional_kwargs={}, example=False)